In [66]:
from algo_module import Algo_manager
from datetime import datetime, timedelta
from time import perf_counter
from time import sleep
import json
import os
import warnings
from kiteconnect import KiteConnect
warnings.filterwarnings("ignore")
from broker_module import Broker
import pandas as pd
from IPython.display import display
pd.options.display.max_columns = None

In [68]:
def generate_access_token(broker_secret, request_token) -> str:

        kite = KiteConnect(api_key=broker_secret['kite_api_key'])

        data = kite.generate_session(request_token=request_token, \
            api_secret=broker_secret['kite_api_secret'])
        print("Token Generated")
        del kite 

        return data['access_token']


def generate_algo_manager (**kwargs):

    parent = os.path.dirname(os.getcwd())
    file_path = os.path.join(parent,kwargs['broker_secret_file_name'])

    with open (file_path, "r") as openfile:
        broker_secret = json.load(openfile)

    if not kwargs['is_kite_access_token_available']:
        broker_secret['kite_access_token'] = \
            generate_access_token(broker_secret,kwargs['kite_request_token'])

        with open (file_path, "w") as outfile:
            json.dump(broker_secret,outfile)

    logs_folder_path = kwargs['log_folder_name']
    # logs_folder_path = os.path.join(parent,kwargs['log_folder_name'])

    if kwargs['broker_for_data'].upper() == 'SIM':
        current_datetime = kwargs['day_start_datetime']
    else: 
        current_datetime = datetime.now()

    module_initiation_time = datetime.now()
    algo_manager = Algo_manager(
        broker_for_trade=kwargs['broker_for_trade'],
        broker_for_data=kwargs['broker_for_data'],
        per_trade_fee = kwargs['per_trade_fee'],
        underlying_name=kwargs['underlying_name'],
        kotak_consumer_key=broker_secret['kotak_consumer_key'],
        kotak_access_token=broker_secret['kotak_access_token'],
        kotak_consumer_secret=broker_secret['kotak_consumer_secret'],
        kotak_user_id=broker_secret['kotak_user_id'],
        kotak_access_code=broker_secret['kotak_access_code'],
        kotak_user_password=broker_secret['kotak_user_password'],
        kite_api_key=broker_secret['kite_api_key'],
        kite_access_token=broker_secret['kite_access_token'],
        log_folder=logs_folder_path,
        current_datetime = current_datetime,
        broker_connection_loss = kwargs['broker_connection_loss'],
        exchange_connection_loss = kwargs['exchange_connection_loss'],
        begin_time=kwargs['trading_start_time'],
        close_time=kwargs['trading_close_time'],
        quantity_per_lot = 50,
        lots_traded = kwargs['lots_traded'],
        total_loss_limit_per_lot = -1_500,
        max_trailing_loss_non_expiry_per_lot = -250,
        max_trailing_loss_expiry_per_lot = -200,
        trailing_loss_trigger_per_lot = 1_500,
        non_expiry_day_no_candle_time = kwargs['non_expiry_day_no_candle_time'],
        expiry_day_no_candle_time = kwargs['expiry_day_no_candle_time'],
        candle_length=kwargs['candle_length'],
        historical_data_folder_name = kwargs['historical_data_folder_name'],
        fno_folder_name = kwargs['fno_folder_name'],
        equity_folder_name = kwargs['equity_folder_name']
        )
    print(f'Module Initiation took: {datetime.now()-module_initiation_time}')

    return algo_manager

In [69]:
#For Live Trading
# day_start_datetime = None
# trading_start_time = datetime(2020,1,1,9,58).time()
# trading_close_time = datetime(2020,1,1,15,7).time()
# switch_off_time = datetime(2020,1,1,15,10,0).time()

#For Simulation
# day_start_datetime = datetime(2021,5,17,9,15)
# trading_start_time = datetime(2021,5,17,9,20).time()
# trading_close_time = datetime(2021,5,17,9,25).time()
# switch_off_time =    datetime(2021,5,17,9,25).time()

# For Live Paper trade
day_start_datetime = None
trading_start_time = datetime(2020,1,1,12,0).time()
trading_close_time = datetime(2020,1,1,15,7).time()
switch_off_time =    datetime(2020,1,1,15,10).time()

# non_expiry_day_no_candle_time = datetime(2020, 1, 1, 9, 15).time()
non_expiry_day_no_candle_time = datetime(2020, 1, 1, 14, 30).time()
expiry_day_no_candle_time = datetime(2020, 1, 1, 13, 0).time()

is_kite_access_token_available = False
kite_request_token='Qe3MLuYU5lkbBWI2PnSaozfe5PCYeO5G'

broker_secret_file_name = 'broker_secret.json'

log_folder_name = 'logs'

candle_length = 5
per_trade_fee = -20
lots_traded = 1
underlying_name = 'NIFTY'

broker_for_trade = 'zerodha'
broker_for_data = 'zerodha'

pause_between_iterations = .7 

# broker_connection_loss = [{'start_datetime':datetime(2021,5,17,9,16),
#                             'end_datetime':datetime(2021,5,17,9,18)},
#                         {'start_datetime':datetime(2021,5,17,9,22),
#                         'end_datetime':datetime(2021,5,17,9,23)},
#                         {'start_datetime':datetime(2021,5,17,10,22),
#                         'end_datetime':datetime(2021,5,17,10,23)}]

broker_connection_loss = None
exchange_connection_loss = None

historical_data_folder_name = 'historical data'
fno_folder_name = 'FNO'
equity_folder_name = 'Equity'

algo_manager = generate_algo_manager (day_start_datetime=day_start_datetime,
                    trading_start_time=trading_start_time,
                    trading_close_time = trading_close_time,
                    switch_off_time = switch_off_time,
                    non_expiry_day_no_candle_time = non_expiry_day_no_candle_time,
                    expiry_day_no_candle_time = expiry_day_no_candle_time,
                    is_kite_access_token_available = is_kite_access_token_available,
                    kite_request_token = kite_request_token,
                    broker_secret_file_name = broker_secret_file_name,
                    log_folder_name = log_folder_name,
                    candle_length = candle_length,
                    per_trade_fee = per_trade_fee,
                    lots_traded = lots_traded,
                    broker_for_trade = broker_for_trade,
                    broker_for_data = broker_for_data,
                    underlying_name = underlying_name,
                    pause_between_iterations = pause_between_iterations,
                    historical_data_folder_name = historical_data_folder_name,
                    fno_folder_name = fno_folder_name,
                    equity_folder_name = equity_folder_name,
                    broker_connection_loss = broker_connection_loss,
                    exchange_connection_loss = exchange_connection_loss)

Token Generated
Module Initiation took: 0:00:01.635779


In [39]:
# end_time = datetime(2021,5,11,14,38)
current_datetime = datetime.now()
initiation_time = perf_counter()
expiry_datetime  = datetime(2022,2,17,15,30)

instrument_id = algo_manager.broker.get_fno_instrument_id(broker_for='trade',strike=17500,underlying='NIFTY',call_put='CE',expiry_datetime=expiry_datetime)
instrument_id_1 = algo_manager.broker.get_fno_instrument_id(broker_for='trade',strike=17500,underlying='NIFTY',call_put='CE',expiry_datetime=expiry_datetime)
instrument_id_2 = algo_manager.broker.get_fno_instrument_id(broker_for='trade',strike=17400,underlying='NIFTY',call_put='CE',expiry_datetime=expiry_datetime)

instruments_df = pd.DataFrame({'instrument_id_data':[instrument_id_1,instrument_id_2]})

# id = algo_manager.broker.place_market_order(instrument_id=instrument_id,buy_sell='buy',quantity=50,current_datetime=current_datetime,initiation_time=initiation_time)

# cancel = algo_manager.broker.get(broker_order_id=instrument_id,current_datetime=current_datetime)

# ltp = algo_manager.broker.get_ltp(instrument_id=instrument_id,current_datetime=current_datetime,initiation_time=initiation_time, exchange='NFO')

# ltp = algo_manager.broker.get_multiple_ltp(instruments_df=instruments_df,current_datetime=current_datetime,initiation_time=initiation_time, exchange='NFO')

# positions = algo_manager.broker.get_positions(current_datetime=current_datetime)

pnl = algo_manager.broker.get_pnl(current_datetime=current_datetime,initiation_time=initiation_time)

# order = algo_manager.broker.is_order_complete(broker_order_id=instrument_id_1,current_datetime=current_datetime)

print(pnl)


None


In [37]:
df = pd.DataFrame({'a':[1,2,3],'b':[5,7,9]})
df['c'] = None
df['d'] = df['c'] + df['b']
df['d'].sum()

0

In [90]:
pdf = pd.DataFrame(p['day'])
pdf

,tradingsymbol,exchange,instrument_token,product,quantity,overnight_quantity,multiplier,average_price,close_price,last_price,value,pnl,m2m,unrealised,realised,buy_quantity,buy_price,buy_value,buy_m2m,sell_quantity,sell_price,sell_value,sell_m2m,day_buy_quantity,day_buy_price,day_buy_value,day_sell_quantity,day_sell_price,day_sell_value
0,NIFTY2221716600PE,NFO,13526530,MIS,50,0,1,11.2375,0,11.05,-590.0,-37.5,-37.5,-37.5,0,200,11.2375,2247.5,2247.5,150,11.05,1657.5,1657.5,200,11.2375,2247.5,150,11.05,1657.5
1,NIFTY2221716700PE,NFO,13527554,MIS,50,0,1,16.4500,0,14.45,-852.5,-130.0,-130.0,-130.0,0,100,16.4500,1645.0,1645.0,50,15.85,792.5,792.5,100,16.4500,1645.0,50,15.85,792.5
2,NIFTY2221717800CE,NFO,13618946,MIS,50,0,1,19.2000,0,18.15,-960.0,-52.5,-52.5,-52.5,0,50,19.2000,960.0,960.0,0,0.00,0.0,0.0,50,19.2000,960.0,0,0.00,0.0
3,NIFTY2221718100CE,NFO,13644546,MIS,0,0,1,0.0000,0,3.10,-7.5,-7.5,-7.5,-7.5,0,50,3.7000,185.0,185.0,50,3.55,177.5,177.5,50,3.7000,185.0,50,3.55,177.5


In [72]:
p['qty_in_hand'] = p['buy_quantity'] - p ['sell_quantity']
p['cash_flow_realized'] = (p['sell_quantity']*p['sell_price']) - (p['buy_quantity'] * p['buy_price'])
p['cash_flow_unrealized'] = p['qty_in_hand'] * p['last_price']
p['cash_flow_realized'].sum() + p['cash_flow_unrealized'].sum()

-87.5

In [74]:
a = algo_manager.broker.get_positions()
print(a)

None


In [65]:
s = None

if s is None:
    print("Yes")


Yes


In [57]:
df = pd.DataFrame({"index":{"0":0,"1":0,"2":0,"3":0,"4":0,"5":0,"6":0,"7":0,"8":0,"9":0,"10":0,"11":0,"12":0,"13":0,"14":0,"15":0,"16":0,"17":0,"18":0,"19":0,"20":0,"21":0,"22":0,"23":0,"24":0,"25":0,"26":0,"27":0,"28":0,"29":0,"30":0,"31":0,"32":0,"33":0,"34":0,"35":0,"36":0,"37":0,"38":0,"39":0,"40":0,"41":0,"42":0,"43":0,"44":0,"45":0,"46":0,"47":0,"48":0,"49":0,"50":0,"51":0,"52":0,"53":0,"54":0},"underlying":{"0":"NIFTY","1":"NIFTY","2":"NIFTY","3":"NIFTY","4":"NIFTY","5":"NIFTY","6":"NIFTY","7":"NIFTY","8":"NIFTY","9":"NIFTY","10":"NIFTY","11":"NIFTY","12":"NIFTY","13":"NIFTY","14":"NIFTY","15":"NIFTY","16":"NIFTY","17":"NIFTY","18":"NIFTY","19":"NIFTY","20":"NIFTY","21":"NIFTY","22":"NIFTY","23":"NIFTY","24":"NIFTY","25":"NIFTY","26":"NIFTY","27":"NIFTY","28":"NIFTY","29":"NIFTY","30":"NIFTY","31":"NIFTY","32":"NIFTY","33":"NIFTY","34":"NIFTY","35":"NIFTY","36":"NIFTY","37":"NIFTY","38":"NIFTY","39":"NIFTY","40":"NIFTY","41":"NIFTY","42":"NIFTY","43":"NIFTY","44":"NIFTY","45":"NIFTY","46":"NIFTY","47":"NIFTY","48":"NIFTY","49":"NIFTY","50":"NIFTY","51":"NIFTY","52":"NIFTY","53":"NIFTY","54":"NIFTY"},"call_put":{"0":"CE","1":"CE","2":"CE","3":"CE","4":"CE","5":"CE","6":"CE","7":"CE","8":"CE","9":"CE","10":"CE","11":"CE","12":"CE","13":"CE","14":"CE","15":"CE","16":"CE","17":"CE","18":"CE","19":"CE","20":"CE","21":"CE","22":"CE","23":"CE","24":"CE","25":"CE","26":"CE","27":"CE","28":"CE","29":"CE","30":"CE","31":"CE","32":"CE","33":"CE","34":"CE","35":"CE","36":"CE","37":"CE","38":"CE","39":"CE","40":"CE","41":"CE","42":"CE","43":"CE","44":"CE","45":"CE","46":"CE","47":"CE","48":"CE","49":"CE","50":"CE","51":"CE","52":"CE","53":"CE","54":"CE"},"expiry_datetime":{"0":1621524600000,"1":1621524600000,"2":1621524600000,"3":1621524600000,"4":1621524600000,"5":1621524600000,"6":1621524600000,"7":1621524600000,"8":1621524600000,"9":1621524600000,"10":1621524600000,"11":1621524600000,"12":1621524600000,"13":1621524600000,"14":1621524600000,"15":1621524600000,"16":1621524600000,"17":1621524600000,"18":1621524600000,"19":1621524600000,"20":1621524600000,"21":1621524600000,"22":1621524600000,"23":1621524600000,"24":1621524600000,"25":1621524600000,"26":1621524600000,"27":1621524600000,"28":1621524600000,"29":1621524600000,"30":1621524600000,"31":1621524600000,"32":1621524600000,"33":1621524600000,"34":1621524600000,"35":1621524600000,"36":1621524600000,"37":1621524600000,"38":1621524600000,"39":1621524600000,"40":1621524600000,"41":1621524600000,"42":1621524600000,"43":1621524600000,"44":1621524600000,"45":1621524600000,"46":1621524600000,"47":1621524600000,"48":1621524600000,"49":1621524600000,"50":1621524600000,"51":1621524600000,"52":1621524600000,"53":1621524600000,"54":1621524600000},"strike":{"0":16000.0,"1":14850.0,"2":15750.0,"3":14600.0,"4":15500.0,"5":14350.0,"6":16400.0,"7":15250.0,"8":14100.0,"9":16150.0,"10":15000.0,"11":13850.0,"12":15900.0,"13":14750.0,"14":15650.0,"15":14500.0,"16":15400.0,"17":14250.0,"18":16300.0,"19":15150.0,"20":14000.0,"21":16050.0,"22":14900.0,"23":15800.0,"24":14650.0,"25":13500.0,"26":15550.0,"27":14400.0,"28":16450.0,"29":15300.0,"30":14150.0,"31":13000.0,"32":16200.0,"33":15050.0,"34":13900.0,"35":15950.0,"36":14800.0,"37":15700.0,"38":14550.0,"39":15450.0,"40":14300.0,"41":16350.0,"42":15200.0,"43":14050.0,"44":16100.0,"45":14950.0,"46":13800.0,"47":15850.0,"48":14700.0,"49":15600.0,"50":14450.0,"51":15350.0,"52":14200.0,"53":16250.0,"54":15100.0},"instrument_id_trade":{"0":"NIFTY20MAY2116000CE.NFO","1":"NIFTY20MAY2114850CE.NFO","2":"NIFTY20MAY2115750CE.NFO","3":"NIFTY20MAY2114600CE.NFO","4":"NIFTY20MAY2115500CE.NFO","5":"NIFTY20MAY2114350CE.NFO","6":"NIFTY20MAY2116400CE.NFO","7":"NIFTY20MAY2115250CE.NFO","8":"NIFTY20MAY2114100CE.NFO","9":"NIFTY20MAY2116150CE.NFO","10":"NIFTY20MAY2115000CE.NFO","11":"NIFTY20MAY2113850CE.NFO","12":"NIFTY20MAY2115900CE.NFO","13":"NIFTY20MAY2114750CE.NFO","14":"NIFTY20MAY2115650CE.NFO","15":"NIFTY20MAY2114500CE.NFO","16":"NIFTY20MAY2115400CE.NFO","17":"NIFTY20MAY2114250CE.NFO","18":"NIFTY20MAY2116300CE.NFO","19":"NIFTY20MAY2115150CE.NFO","20":"NIFTY20MAY2114000CE.NFO","21":"NIFTY20MAY2116050CE.NFO","22":"NIFTY20MAY2114900CE.NFO","23":"NIFTY20MAY2115800CE.NFO","24":"NIFTY20MAY2114650CE.NFO","25":"NIFTY20MAY2113500CE.NFO","26":"NIFTY20MAY2115550CE.NFO","27":"NIFTY20MAY2114400CE.NFO","28":"NIFTY20MAY2116450CE.NFO","29":"NIFTY20MAY2115300CE.NFO","30":"NIFTY20MAY2114150CE.NFO","31":"NIFTY20MAY2113000CE.NFO","32":"NIFTY20MAY2116200CE.NFO","33":"NIFTY20MAY2115050CE.NFO","34":"NIFTY20MAY2113900CE.NFO","35":"NIFTY20MAY2115950CE.NFO","36":"NIFTY20MAY2114800CE.NFO","37":"NIFTY20MAY2115700CE.NFO","38":"NIFTY20MAY2114550CE.NFO","39":"NIFTY20MAY2115450CE.NFO","40":"NIFTY20MAY2114300CE.NFO","41":"NIFTY20MAY2116350CE.NFO","42":"NIFTY20MAY2115200CE.NFO","43":"NIFTY20MAY2114050CE.NFO","44":"NIFTY20MAY2116100CE.NFO","45":"NIFTY20MAY2114950CE.NFO","46":"NIFTY20MAY2113800CE.NFO","47":"NIFTY20MAY2115850CE.NFO","48":"NIFTY20MAY2114700CE.NFO","49":"NIFTY20MAY2115600CE.NFO","50":"NIFTY20MAY2114450CE.NFO","51":"NIFTY20MAY2115350CE.NFO","52":"NIFTY20MAY2114200CE.NFO","53":"NIFTY20MAY2116250CE.NFO","54":"NIFTY20MAY2115100CE.NFO"},"instrument_id_data":{"0":"NIFTY20MAY2116000CE.NFO","1":"NIFTY20MAY2114850CE.NFO","2":"NIFTY20MAY2115750CE.NFO","3":"NIFTY20MAY2114600CE.NFO","4":"NIFTY20MAY2115500CE.NFO","5":"NIFTY20MAY2114350CE.NFO","6":"NIFTY20MAY2116400CE.NFO","7":"NIFTY20MAY2115250CE.NFO","8":"NIFTY20MAY2114100CE.NFO","9":"NIFTY20MAY2116150CE.NFO","10":"NIFTY20MAY2115000CE.NFO","11":"NIFTY20MAY2113850CE.NFO","12":"NIFTY20MAY2115900CE.NFO","13":"NIFTY20MAY2114750CE.NFO","14":"NIFTY20MAY2115650CE.NFO","15":"NIFTY20MAY2114500CE.NFO","16":"NIFTY20MAY2115400CE.NFO","17":"NIFTY20MAY2114250CE.NFO","18":"NIFTY20MAY2116300CE.NFO","19":"NIFTY20MAY2115150CE.NFO","20":"NIFTY20MAY2114000CE.NFO","21":"NIFTY20MAY2116050CE.NFO","22":"NIFTY20MAY2114900CE.NFO","23":"NIFTY20MAY2115800CE.NFO","24":"NIFTY20MAY2114650CE.NFO","25":"NIFTY20MAY2113500CE.NFO","26":"NIFTY20MAY2115550CE.NFO","27":"NIFTY20MAY2114400CE.NFO","28":"NIFTY20MAY2116450CE.NFO","29":"NIFTY20MAY2115300CE.NFO","30":"NIFTY20MAY2114150CE.NFO","31":"NIFTY20MAY2113000CE.NFO","32":"NIFTY20MAY2116200CE.NFO","33":"NIFTY20MAY2115050CE.NFO","34":"NIFTY20MAY2113900CE.NFO","35":"NIFTY20MAY2115950CE.NFO","36":"NIFTY20MAY2114800CE.NFO","37":"NIFTY20MAY2115700CE.NFO","38":"NIFTY20MAY2114550CE.NFO","39":"NIFTY20MAY2115450CE.NFO","40":"NIFTY20MAY2114300CE.NFO","41":"NIFTY20MAY2116350CE.NFO","42":"NIFTY20MAY2115200CE.NFO","43":"NIFTY20MAY2114050CE.NFO","44":"NIFTY20MAY2116100CE.NFO","45":"NIFTY20MAY2114950CE.NFO","46":"NIFTY20MAY2113800CE.NFO","47":"NIFTY20MAY2115850CE.NFO","48":"NIFTY20MAY2114700CE.NFO","49":"NIFTY20MAY2115600CE.NFO","50":"NIFTY20MAY2114450CE.NFO","51":"NIFTY20MAY2115350CE.NFO","52":"NIFTY20MAY2114200CE.NFO","53":"NIFTY20MAY2116250CE.NFO","54":"NIFTY20MAY2115100CE.NFO"},"instrument_ltp":{"0":None,"1":None,"2":None,"3":None,"4":None,"5":None,"6":None,"7":None,"8":None,"9":None,"10":None,"11":None,"12":None,"13":None,"14":None,"15":None,"16":None,"17":None,"18":None,"19":None,"20":None,"21":None,"22":None,"23":None,"24":None,"25":None,"26":None,"27":None,"28":None,"29":None,"30":None,"31":None,"32":None,"33":None,"34":None,"35":None,"36":None,"37":None,"38":None,"39":None,"40":None,"41":None,"42":None,"43":None,"44":None,"45":None,"46":None,"47":None,"48":None,"49":None,"50":None,"51":None,"52":None,"53":None,"54":None}})
df

,index,underlying,call_put,expiry_datetime,strike,instrument_id_trade,instrument_id_data,instrument_ltp
0,0,NIFTY,CE,1621524600000,16000.0,NIFTY20MAY2116000CE.NFO,NIFTY20MAY2116000CE.NFO,None
1,0,NIFTY,CE,1621524600000,14850.0,NIFTY20MAY2114850CE.NFO,NIFTY20MAY2114850CE.NFO,None
2,0,NIFTY,CE,1621524600000,15750.0,NIFTY20MAY2115750CE.NFO,NIFTY20MAY2115750CE.NFO,None
3,0,NIFTY,CE,1621524600000,14600.0,NIFTY20MAY2114600CE.NFO,NIFTY20MAY2114600CE.NFO,None
4,0,NIFTY,CE,1621524600000,15500.0,NIFTY20MAY2115500CE.NFO,NIFTY20MAY2115500CE.NFO,None
5,0,NIFTY,CE,1621524600000,14350.0,NIFTY20MAY2114350CE.NFO,NIFTY20MAY2114350CE.NFO,None
6,0,NIFTY,CE,1621524600000,16400.0,NIFTY20MAY2116400CE.NFO,NIFTY20MAY2116400CE.NFO,None
7,0,NIFTY,CE,1621524600000,15250.0,NIFTY20MAY2115250CE.NFO,NIFTY20MAY2115250CE.NFO,None
8,0,NIFTY,CE,1621524600000,14100.0,NIFTY20MAY2114100CE.NFO,NIFTY20MAY2114100CE.NFO,None
9,0,NIFTY,CE,1621524600000,16150.0,NIFTY20MAY2116150CE.NFO,NIFTY20MAY2116150CE.NFO,None
